In [44]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import tensorflow as tf

# Initialize the webcam and MediaPipe Pose
cap = cv2.VideoCapture(0)  # '0' for the primary camera
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

evaluate = True

# Load model
model = tf.keras.models.load_model("best_model_grid_search.h5")

# Data storage
landmarks_data = []
labels = []
map = {0: "squat_up", 1: "squat_down", 2: 'jumping_jack_down', 3: 'jumping_jack_up', 4: 'curl_down', 5: 'curl_up', 6: 'lunge_up', 7: 'lunge_down', 8: 'high_knee_down', 9: 'high_knee_up', 10: 'push_up_up', 11: 'push_up_down', 12: 'back_row_down', 13: 'back_row_up'}

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Pose
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        # Draw the pose annotations on the frame
        mp.solutions.drawing_utils.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
        )

        # Extract landmarks
        landmarks = (
            [landmark.x for landmark in results.pose_landmarks.landmark]
            + [landmark.y for landmark in results.pose_landmarks.landmark]
            + [landmark.z for landmark in results.pose_landmarks.landmark]
        )

        # Flatten the landmarks into a single array
        landmarks = np.array(landmarks).flatten()

        if evaluate:
            landmarks = landmarks.reshape(1, -1)
            # Make a prediction using the __call__ method for a single batch
            prediction = model(landmarks, training=False)  # Set training to False for inference mode

            # Convert prediction to numpy array if it's not already (for TensorFlow 2.x)
            prediction = prediction.numpy()

            # Get the index of the max confidence
            predicted_index = np.argmax(prediction)

            # Get the max confidence value
            confidence = np.max(prediction)

            # Check if the confidence is above the threshold
            if confidence > 0.5:
                # Display the class with the highest confidence
                label = map[predicted_index]
            else:
                # Display 'Unknown' if confidence is not high enough
                label = "Unknown"

            # Put the text on the frame
            cv2.putText(frame, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    # Display the frame
    cv2.imshow("Pose Detection", frame)

    # Wait for key press
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
    elif ord("0") <= key <= ord("9"):
        # If a number key is pressed, save the current landmarks and the pressed key as a label
        if results.pose_landmarks:
            landmarks_data.append(landmarks)
            labels.append(key - ord("0"))  # Convert ASCII value to integer (0-9)
            print(f"Landmarks and label saved: {key - ord('0')}")

# Release the webcam and destroy all OpenCV windows
cap.release()
cv2.destroyAllWindows()

# Convert the collected data to a DataFrame
import os
df_data = pd.DataFrame(landmarks_data)
df_data["label"] = labels  # Add the labels as the last column
file_exists = os.path.isfile("landmarks_and_labels.csv")
# Save the DataFrame to a CSV file
if not evaluate:
    df_data.to_csv(
    "landmarks_and_labels.csv", mode="a", index=False, header=not file_exists
)

    print("Data saved to 'landmarks_and_labels.csv'")